In [ ]:
import ee
import ee.batch
import pyperclip

In [ ]:
# Initialize the Earth Engine library.
ee.Authenticate()
ee.Initialize(project='pixelcampione-satelital2map')


In [52]:
import ee.data


satellital_ids = []
elevation_ids = []

In [ ]:
import ee.batch


def export_elevation(dataset_image,area_of_interest, filename, copy: bool = True):
    dataset = ee.image.Image(dataset_image)

    selected = dataset.select('elevation')
    # Define export parameters.
    task = ee.batch.Export.image.toDrive(**{
        'image': selected,
        'description': 'elevation_to_drive',
        'fileNamePrefix': filename,
        'scale': 90,  # Adjust scale if necessary
        'region': area_of_interest,
        'fileFormat': 'GeoTIFF',
        'folder': 'dataset'  # Specify the Google Drive folder name here.
    })

    # Start the export task.
    task.start()

    # Print the task ID
    print(f'Task ID: {task.id}')
    elevation_ids.append(task.id)


def export_rgb(image: ee.image.Image,area_of_interest, filename, copy: bool = True):
        
    # Select the RGB bands
    selected = image.select(['B4', 'B3', 'B2'])

    # Define export parameters.
    task = ee.batch.Export.image.toDrive(**{
        'image': selected,
        'description':'rgb_to_drive',
        'fileNamePrefix': filename,
        'scale': 30,  # Adjust scale if necessary
        'region': area_of_interest,
        'fileFormat': 'GeoTIFF',
        'folder': 'dataset'  # Specify the Google Drive folder name here.
    })

    # Start the export task.
    task.start()

    # Print the task ID
    print(f'Task ID: {task.id}')
    satellital_ids.append(task.id)

In [ ]:
center = {
    'longitude': -120.0,
    'latitude': 40.0
}
offset = 0.10

# 5 x 5
lat_mult = 3
lon_mult = 3
index = {
    'start_lat': 6*lat_mult,
    'start_lon': 6*lon_mult,
    'end_lat': 6*(lat_mult+1),
    'end_lon': 6*(lon_mult+1)
}
# Me quede a partir 361 , incluyendolo
temp = 649

In [ ]:
counter = temp
for i in range(index['start_lat'],index['end_lat']):
    for j in range(index['start_lon'], index['end_lon']):
        # Define an area of interest (for example, a bounding box around Mt. Everest).
        start = ee.geometry.Geometry.Rectangle([center['longitude'] - offset + 2*offset*j, center['latitude'] - offset - 2*offset*i
                                                , center['longitude'] + offset+ 2*offset*j, center['latitude'] + offset - 2*offset*i])
        # Call the function with your area of interest and desired filename prefix.
        export_elevation("CGIAR/SRTM90_V4",start, f'elevation_{counter}')
        counter += 1

        print("j = ",j)
    print("i = ",i)

print("-------------------------ELEVATION FINISHED-------------------------")

In [ ]:
counter = temp
for i in range(index['start_lat'],index['end_lat']):
    for j in range(index['start_lon'], index['end_lon']):
        # Define an area of interest (for example, a bounding box around Mt. Everest).
        start = ee.geometry.Geometry.Rectangle([center['longitude'] - offset + 2*offset*j, center['latitude'] - offset + 2*offset*i
                                                , center['longitude'] + offset+ 2*offset*j, center['latitude'] + offset + 2*offset*i])

        # Convert ImageCollection to Image
        image = ee.imagecollection.ImageCollection("LANDSAT/LC09/C02/T1").filterBounds(start).mosaic()
        
        # Call the function with your area of interest and desired filename prefix.
        export_rgb(image, start, f'satellite_{counter}')
        counter += 1
        

        print("j = ",j)
    print("i = ",i)

print("-------------------------SATELLITE FINISHED-------------------------")

In [ ]:
print("Export finished!")
pyperclip.copy("\'satellital\': " + str(satellital_ids)+"," +"\n"
               "\'elevation\': " + str(elevation_ids) + "\n")
